# Primer Entregable: Proceso ETL en API
Alumna: Ruth Cruz
Comisión 56035



In [86]:
#Importamos las librerias necesarias para realizar el proyecto
import requests
import pandas as pd
import json as json
from datetime import datetime , timedelta
from configparser import ConfigParser

In [87]:
#la api seleccionada muestra los datos actuales de personas contagiadas de Covid en todo el territorio de EEUU, (dividido por estados  y condaddos )
url = "https://api.covidactnow.org/v2/counties.json?apiKey=14c3113b02cb4cca87736b9f60a1f4e7"

In [88]:
params = {
    'lastUpdatedDate': (datetime.utcnow() - timedelta(hours=20)).strftime('%Y-%m-%dT%H:00:00'),
    'end': (datetime.utcnow() - timedelta(hours=20)).strftime('%Y-%m-%dT%H:59:59')
}
params

{'lastUpdatedDate': '2024-02-27T01:00:00', 'end': '2024-02-27T01:59:59'}

In [89]:
#comprobamos la conexion a la API
r = requests.get(url, params=params)
r.raise_for_status()

In [90]:
# Visualizando los datos crudos
data = r.json()
df = pd.json_normalize(data)
df.head()

,fips,country,state,county,hsa,hsaName,level,lat,locationId,long,...,actuals.vaccinationsInitiatedDemographics.age.65-79,actuals.vaccinationsInitiatedDemographics.age.12-24,actuals.vaccinationsInitiatedDemographics.age.16-20,actuals.vaccinationsInitiatedDemographics.age.21-25,actuals.vaccinationsInitiatedDemographics.age.26-30,actuals.vaccinationsInitiatedDemographics.age.31-40,actuals.vaccinationsInitiatedDemographics.age.41-50,actuals.vaccinationsInitiatedDemographics.age.51-60,actuals.vaccinationsInitiatedDemographics.age.61-70,actuals.vaccinationsInitiatedDemographics.age.71_plus
0,02013,US,AK,Aleutians East Borough,820,"Anchorage (Anchorage), AK - Matanuska-Susitna ...",county,None,iso1:us#iso2:us-ak#fips:02013,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02016,US,AK,Aleutians West Census Area,820,"Anchorage (Anchorage), AK - Matanuska-Susitna ...",county,None,iso1:us#iso2:us-ak#fips:02016,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02020,US,AK,Anchorage Municipality,820,"Anchorage (Anchorage), AK - Matanuska-Susitna ...",county,None,iso1:us#iso2:us-ak#fips:02020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,02050,US,AK,Bethel Census Area,820,"Anchorage (Anchorage), AK - Matanuska-Susitna ...",county,None,iso1:us#iso2:us-ak#fips:02050,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,02060,US,AK,Bristol Bay Borough,820,"Anchorage (Anchorage), AK - Matanuska-Susitna ...",county,None,iso1:us#iso2:us-ak#fips:02060,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Segundo Entregable: Proceso ETL en API
Alumna: Ruth Cruz
Comisión 56035



Limpiando los datos, eliminando columnas innecesarias y trabajando los valores nulos

In [91]:
!pip install psycopg2

In [92]:
# eliminando columnas innecesarias
eliminar = [
    'annotations.vaccinationsFall2022BivalentBooster',
    'actuals.vaccinationsInitiatedDemographics.age.0-17',
    'actuals.vaccinationsInitiatedDemographics.age.0-4',
    'actuals.vaccinationsInitiatedDemographics.age.12-17',
    'actuals.vaccinationsInitiatedDemographics.age.16-54',
    'actuals.vaccinationsInitiatedDemographics.age.18-24',
    'actuals.vaccinationsInitiatedDemographics.age.18-29',
    'actuals.vaccinationsInitiatedDemographics.age.25-49',
    'actuals.vaccinationsInitiatedDemographics.age.30-49',
    'actuals.vaccinationsInitiatedDemographics.age.5-11',
    'actuals.vaccinationsInitiatedDemographics.age.50-64',
    'actuals.vaccinationsInitiatedDemographics.age.55-64',
    'actuals.vaccinationsInitiatedDemographics.age.65-74',
    'actuals.vaccinationsInitiatedDemographics.age.75_plus',
    'actuals.vaccinationsInitiatedDemographics.race.ai_an',
    'actuals.vaccinationsInitiatedDemographics.race.asian',
    'actuals.vaccinationsInitiatedDemographics.race.black',
    'actuals.vaccinationsInitiatedDemographics.race.multiple',
    'actuals.vaccinationsInitiatedDemographics.race.other',
    'actuals.vaccinationsInitiatedDemographics.race.pacific_islander',
    'actuals.vaccinationsInitiatedDemographics.race.unknown',
    'actuals.vaccinationsInitiatedDemographics.race.white',
    'actuals.vaccinationsInitiatedDemographics.ethnicity',
    'actuals.vaccinationsInitiatedDemographics.sex.female',
    'actuals.vaccinationsInitiatedDemographics.sex.male',
    'actuals.vaccinationsInitiatedDemographics.sex.unknown',
    'actuals.vaccinationsInitiatedDemographics.age',
    'actuals.vaccinationsInitiatedDemographics.race',
    'actuals.vaccinationsInitiatedDemographics.ethnicity.hispanic',
    'actuals.vaccinationsInitiatedDemographics.sex',
    'actuals.vaccinationsInitiatedDemographics.race.asian_or_pacific_islander',
    'actuals.vaccinationsInitiatedDemographics.age.18-49',
    'actuals.vaccinationsInitiatedDemographics.age.65_plus',
    'actuals.vaccinationsInitiatedDemographics.race.latino',
    'annotations.vaccinesAdministered.sources',
    'annotations.vaccinesAdministered.anomalies',
    'actuals.vaccinationsInitiatedDemographics.age.18-34',
    'actuals.vaccinationsInitiatedDemographics.age.35-49',
    'actuals.vaccinationsInitiatedDemographics.age.unknown',
    'actuals.vaccinationsInitiatedDemographics.ethnicity.non-hispanic',
    'actuals.vaccinationsInitiatedDemographics.ethnicity.unknown',
    'actuals.vaccinationsInitiatedDemographics.age.10-14',
    'actuals.vaccinationsInitiatedDemographics.age.15-19',
    'actuals.vaccinationsInitiatedDemographics.age.20-24',
    'actuals.vaccinationsInitiatedDemographics.age.25-34',
    'actuals.vaccinationsInitiatedDemographics.age.35-44',
    'actuals.vaccinationsInitiatedDemographics.age.45-54',
    'actuals.vaccinationsInitiatedDemographics.age.5-9',
    'actuals.vaccinationsInitiatedDemographics.age.75-84',
    'actuals.vaccinationsInitiatedDemographics.age.85_plus',
    'actuals.vaccinationsInitiatedDemographics.age.30-39',
    'actuals.vaccinationsInitiatedDemographics.age.40-49',
    'actuals.vaccinationsInitiatedDemographics.age.5-17',
    'actuals.vaccinationsInitiatedDemographics.age.50-59',
    'actuals.vaccinationsInitiatedDemographics.age.60-69',
    'actuals.vaccinationsInitiatedDemographics.age.70_plus',
    'actuals.vaccinationsInitiatedDemographics.race.middle_eastern_or_north_africa',
    'actuals.vaccinationsInitiatedDemographics.age.0-19',
    'actuals.vaccinationsInitiatedDemographics.age.12-15',
    'actuals.vaccinationsInitiatedDemographics.age.16-19',
    'actuals.vaccinationsInitiatedDemographics.age.20-29',
    'actuals.vaccinationsInitiatedDemographics.race.hispanic',
    'actuals.vaccinationsInitiatedDemographics.sex.other',
    'actuals.vaccinationsInitiatedDemographics.age.0-11',
    'actuals.vaccinationsInitiatedDemographics.age.12-19',
    'actuals.vaccinationsInitiatedDemographics.age.70-79',
    'actuals.vaccinationsInitiatedDemographics.age.80_plus',
    'actuals.vaccinationsInitiatedDemographics.age.16-17',
    'annotations.vaccinesDistributed.sources',
    'annotations.vaccinesDistributed.anomalies',
    'actuals.vaccinationsInitiatedDemographics.age.0-15',
    'actuals.vaccinationsInitiatedDemographics.age.16-25',
    'actuals.vaccinationsInitiatedDemographics.age.18-25',
    'actuals.vaccinationsInitiatedDemographics.age.26-34',
    'actuals.vaccinationsInitiatedDemographics.age.60-64',
    'actuals.vaccinationsInitiatedDemographics.age.65-69',
    'actuals.vaccinationsInitiatedDemographics.age.70-74',
    'actuals.vaccinationsInitiatedDemographics.age.75-79',
    'actuals.vaccinationsInitiatedDemographics.age.18-19',
    'actuals.vaccinationsInitiatedDemographics.age.20-49',
    'actuals.vaccinationsInitiatedDemographics.age.100-104',
    'actuals.vaccinationsInitiatedDemographics.age.25-29',
    'actuals.vaccinationsInitiatedDemographics.age.30-34',
    'actuals.vaccinationsInitiatedDemographics.age.35-39',
    'actuals.vaccinationsInitiatedDemographics.age.40-44',
    'actuals.vaccinationsInitiatedDemographics.age.45-49',
    'actuals.vaccinationsInitiatedDemographics.age.65-79',
    'actuals.vaccinationsInitiatedDemographics.age.12-24',
    'actuals.vaccinationsInitiatedDemographics.age.16-20',
    'actuals.vaccinationsInitiatedDemographics.age.21-25',
    'actuals.vaccinationsInitiatedDemographics.age.26-30',
    'actuals.vaccinationsInitiatedDemographics.age.31-40',
    'actuals.vaccinationsInitiatedDemographics.age.41-50',
    'actuals.vaccinationsInitiatedDemographics.age.51-60',
    'actuals.vaccinationsInitiatedDemographics.age.61-70',
    'actuals.vaccinationsInitiatedDemographics.age.71_plus',
    'actuals.vaccinationsInitiatedDemographics.age.16-49',
    'actuals.vaccinesAdministeredDemographics.age.65-79',
    'actuals.vaccinesAdministeredDemographics.age.80_plus',
    'actuals.vaccinesAdministeredDemographics.age.unknown',
    'actuals.vaccinesAdministeredDemographics.race.asian',
    'actuals.vaccinesAdministeredDemographics.race.black',
    'actuals.vaccinesAdministeredDemographics.race.other',
    'actuals.vaccinesAdministeredDemographics.race.unknown',
    'actuals.vaccinesAdministeredDemographics.race.white',
    'actuals.vaccinesAdministeredDemographics.ethnicity',
    'actuals.vaccinesAdministeredDemographics.sex',
    'actuals.vaccinationsInitiatedDemographics.age.95-99',
    'actuals.vaccinationsInitiatedDemographics.race.multiple_other',
    'actuals.vaccinationsInitiatedDemographics.age.105_plus',
    'actuals.vaccinationsInitiatedDemographics.race.ai_an_asian_or_pacific_islander',
    'actuals.vaccinationsInitiatedDemographics.age.10-19',
    'actuals.vaccinationsInitiatedDemographics.age.0-9',
    'actuals.vaccinesAdministeredDemographics.age.12-15',
    'actuals.vaccinesAdministeredDemographics.age.16-49',
    'actuals.vaccinesAdministeredDemographics.age.5-11',
    'annotations.caseDensity',
    'annotations.weeklyNewCasesPer100k',
    'annotations.infectionRate',
    'annotations.infectionRateCI90',
    'actuals.vaccinationsInitiatedDemographics.age.50-54',
    'actuals.vaccinationsInitiatedDemographics.age.55-59',
    'actuals.vaccinationsInitiatedDemographics.age.80-84',
    'actuals.vaccinationsInitiatedDemographics.age.85-89',
    'actuals.vaccinationsInitiatedDemographics.age.90-94',
    'actuals.vaccinesAdministeredDemographics.age.50-64',
    'annotations.icuCapacityRatio.anomalies',
    'annotations.bedsWithCovidPatientsRatio.sources',
    'annotations.bedsWithCovidPatientsRatio.anomalies',
    'annotations.weeklyCovidAdmissionsPer100k.sources',
    'annotations.weeklyCovidAdmissionsPer100k.anomalies',
    'annotations.vaccinationsInitiated',
    'annotations.vaccinationsCompleted',
    'annotations.vaccinationsAdditionalDose',
    'url',
    'annotations.hospitalBeds.anomalies',
    'annotations.icuBeds.sources',
    'annotations.icuBeds.anomalies',
    'annotations.icuCapacityRatio.sources',
    'annotations.icuCapacityRatio',
    'annotations.bedsWithCovidPatientsRatio',
    'annotations.weeklyCovidAdmissionsPer100k',
    'annotations.vaccinationsInitiatedRatio',
    'annotations.vaccinationsCompletedRatio',
    'annotations.vaccinationsAdditionalDoseRatio',
    'annotations.vaccinationsFall2022BivalentBoosterRatio',
    'annotations.newCases.sources',
    'annotations.newCases.anomalies',
    'annotations.hospitalBeds.sources',
    'locationId',
    'annotations.infectionRate.sources',
    'annotations.infectionRate.anomalies',
    'annotations.infectionRateCI90.sources',
    'annotations.infectionRateCI90.anomalies',
    'annotations.cases.sources',
    'annotations.cases.anomalies',
    'annotations.weeklyNewCasesPer100k.sources',
    'actuals.hospitalBeds.capacity',
    'actuals.hospitalBeds.currentUsageTotal',
    'actuals.hospitalBeds.currentUsageCovid',
    'actuals.hospitalBeds.weeklyCovidAdmissions',
    'actuals.hsaHospitalBeds.capacity',
    'actuals.hsaHospitalBeds.currentUsageTotal',
    'metrics.testPositivityRatioDetails.source',
    'metrics.caseDensity',
    'metrics.weeklyNewCasesPer100k',
    'metrics.contactTracerCapacityRatio',
    'metrics.infectionRate', 'metrics.infectionRateCI90',
    'metrics.icuCapacityRatio', 'metrics.bedsWithCovidPatientsRatio',
    'metrics.weeklyCovidAdmissionsPer100k',
    'metrics.vaccinationsInitiatedRatio',
    'metrics.vaccinationsCompletedRatio',
    'metrics.vaccinationsAdditionalDoseRatio',
    'metrics.vaccinationsFall2022BivalentBoosterRatio',
    'riskLevels.overall',
    'riskLevels.testPositivityRatio',
    'riskLevels.caseDensity',
    'riskLevels.contactTracerCapacityRatio',
    'riskLevels.infectionRate',
    'riskLevels.icuCapacityRatio',
    'communityLevels.cdcCommunityLevel',
    'actuals.vaccinationsInitiated',
    'actuals.vaccinationsCompleted',
    'actuals.vaccinationsAdditionalDose',
    'actuals.vaccinationsFall2022BivalentBooster',
    'actuals.vaccinesAdministered',
    'actuals.vaccinesAdministeredDemographics',
    'actuals.vaccinationsInitiatedDemographics',
    'lat',
    'long',
    'cdcTransmissionLevel',
    'metrics.testPositivityRatio',
    'communityLevels.canCommunityLevel',
    'annotations.vaccinesDistributed',
    'annotations.vaccinationsInitiated.sources',
    'annotations.vaccinationsInitiated.anomalies',
    'annotations.vaccinationsCompleted.sources',
    'annotations.vaccinationsCompleted.anomalies',
    'annotations.vaccinationsAdditionalDose.sources',
    'annotations.vaccinationsAdditionalDose.anomalies',
    'annotations.vaccinationsFall2022BivalentBooster.sources',
    'annotations.vaccinationsFall2022BivalentBooster.anomalies',
    'annotations.vaccinesAdministered',
    'annotations.testPositivityRatio',
    'annotations.caseDensity.sources',
    'annotations.caseDensity.anomalies',
    'annotations.weeklyNewCasesPer100k.anomalies',
    'annotations.contactTracerCapacityRatio',
    'annotations.deaths.sources',
    'annotations.deaths.anomalies',
    'annotations.positiveTests',
    'annotations.negativeTests',
    'annotations.contactTracers',
    'annotations.hospitalBeds',
    'annotations.hsaHospitalBeds',
    'annotations.icuBeds',
    'annotations.hsaIcuBeds',
    'annotations.newCases',
    'annotations.newDeaths',
    'annotations.newDeaths.anomalies',
    'annotations.cases',
    'actuals.vaccinesDistributed',
    'actuals.positiveTests',
    'actuals.negativeTests',
    'actuals.contactTracers',
    'annotations.newDeaths.sources',
    'actuals.hsaHospitalBeds.weeklyCovidAdmissions',
    'hsaName',
    'hsa',
    'hsaPopulation',
    'annotations.deaths',
    'actuals.hsaIcuBeds.capacity',
    'actuals.hsaIcuBeds.currentUsageTotal',
    'actuals.hsaIcuBeds.currentUsageCovid',
    'actuals.newCases',
    'level',
    'actuals.hsaHospitalBeds.currentUsageCovid',
    'actuals.newDeaths'
]

df = df.drop(eliminar, axis=1)

In [93]:
# Trabajando los valores nulos -Estrategia de imputación: Reemplazando con un valor específico
df['actuals.icuBeds.capacity'] = df['actuals.icuBeds.capacity'].fillna(0)  # Reemplaza con 0
df['actuals.icuBeds.currentUsageTotal'] = df['actuals.icuBeds.currentUsageTotal'].fillna(0)  # Reemplaza con 0
df['actuals.icuBeds.currentUsageCovid']= df['actuals.icuBeds.currentUsageCovid'].fillna(0) # Reemplaza con 0

In [94]:
#visulizamos como quedo el df final
df.head(5)

,fips,country,state,county,population,lastUpdatedDate,actuals.cases,actuals.deaths,actuals.icuBeds.capacity,actuals.icuBeds.currentUsageTotal,actuals.icuBeds.currentUsageCovid
0,02013,US,AK,Aleutians East Borough,3337,2024-02-20,812,0,0.0,0.0,0.0
1,02016,US,AK,Aleutians West Census Area,5634,2024-02-20,1174,0,0.0,0.0,0.0
2,02020,US,AK,Anchorage Municipality,288000,2024-02-20,121792,284,63.0,30.0,0.0
3,02050,US,AK,Bethel Census Area,18386,2024-02-20,13328,0,0.0,0.0,0.0
4,02060,US,AK,Bristol Bay Borough,836,2024-02-20,0,0,0.0,0.0,0.0


In [95]:
df.to_csv('data1.csv', index=False)

In [96]:
import psycopg2
from configparser import ConfigParser
import sqlalchemy as sa
from sqlalchemy import create_engine
import pandas as pd

In [97]:
def build_conn_string(config_path, config_section):
    # Lee el archivo de configuración
    parser = ConfigParser()
    parser.read(config_path)

    # Lee la sección de configuración de Redshiftf
    config = parser['redshiftf']  # Usando la sección pasada como argumento
    host = config['host']
    port = config['port']
    dbname = config['dbname']
    username = config['username']
    pwd = config['pwd']

    # Construye la cadena de conexión en formato URL
    conn_string = f"postgresql://{username}:{pwd}@{host}:{port}/{dbname}"
    return conn_string
conn_string = build_conn_string('pipeline.conf', 'redshiftf')

# Crear la conexión y el motor SQLAlchemy
conn = psycopg2.connect(conn_string)


In [98]:
engine = create_engine(conn_string)

In [99]:
#cargamos los datos del df a la tabla
with engine.connect() as conn:
    df.to_sql(
        'covidfinal',
        schema='ruthn_cruz09_coderhouse',
        con=engine,
        index=False,
        method='multi',
        if_exists='append'
    )


ProgrammingError: (psycopg2.errors.UndefinedObject) unrecognized configuration parameter "standard_conforming_strings"

[SQL: show standard_conforming_strings]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
#creando la tabla temporal stg_covidfinal para evitar duplicacion
with engine.connect() as conn:
    conn.execute("""
    CREATE TABLE IF NOT EXISTS stg_covidfinal (
        fips VARCHAR(10),
        country VARCHAR(3),
        state VARCHAR(3),
        county VARCHAR(100),
        population INTEGER,
        lastUpdatedDate TIMESTAMP,
        "actuals.cases" INTEGER,
        "actuals.deaths" INTEGER,
        "actuals.icuBeds.capacity" FLOAT,
        "actuals.icuBeds.currentUsageTotal" FLOAT,
        "actuals.icuBeds.currentUsageCovid" FLOAT,
        PRIMARY KEY (fips, lastUpdatedDate)
    );
    """)


ProgrammingError: (psycopg2.errors.UndefinedObject) unrecognized configuration parameter "standard_conforming_strings"

[SQL: show standard_conforming_strings]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# Truncando la tabla para evitar duplicacion
with engine.connect() as conn, conn.begin():
    conn.execute('TRUNCATE TABLE stg_covidfinal')
    df.to_sql(
        'stg_covidfinal',
        schema='ruthn_cruz09_coderhouse',
        con=engine,
        index=False,
        method='multi',
        if_exists='append'
    )

    conn.execute("""
                MERGE INTO covidfinal
                USING stg_covidfinal AS stg
                ON covidfinal.fips = stg.fips
                AND covidfinal.country = stg.country
                AND covidfinal.state = stg.state
                AND covidfinal.county = stg.county
                AND covidfinal.lastUpdatedDate = stg.lastUpdatedDate
                AND covidfinal."actuals.cases" = stg."actuals.cases"
                AND covidfinal."actuals.deaths" = stg."actuals.deaths"
                AND covidfinal."actuals.icuBeds.capacity" = stg."actuals.icuBeds.capacity"
                WHEN MATCHED THEN
                UPDATE SET population = stg.population
                WHEN NOT MATCHED THEN
                INSERT (fips, country, state, county, population, lastUpdatedDate,
                        "actuals.cases", "actuals.deaths", "actuals.icuBeds.capacity",
                        "actuals.icuBeds.currentUsageTotal", "actuals.icuBeds.currentUsageCovid")
                VALUES (stg.fips, stg.country, stg.state, stg.county, stg.population, stg.lastUpdatedDate,
                        stg."actuals.cases", stg."actuals.deaths", stg."actuals.icuBeds.capacity",
                        stg."actuals.icuBeds.currentUsageTotal", stg."actuals.icuBeds.currentUsageCovid")
                        ;
""")
